In [50]:
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer, confusion_matrix
import matplotlib.pyplot as plt
# The following line is needed to show plots inline in notebooks
%matplotlib inline 

In [51]:
data = pd.read_csv('Kaggle_Salary.csv')
# drop row index 0 because it holds the Questions (inplace is to remove permanently)
data.drop(data.index[0], inplace=True)

In [3]:
# Function to calculate how many missing values are in a column
def how_many_nan(col_name):
    amount = col_name[col_name.isna()].shape[0]
    return amount

In [52]:
# convert salary in to categorical data
data.Q9 = pd.Categorical(data.Q9)

In [53]:
# Q1: Gender
# Combine gender cateogires less than 150 with Other
gender_count = data.Q1.value_counts()
value_mask = data.Q1.isin(gender_count.index[gender_count < 150]) 
data.loc[value_mask,'Q1'] = "Other"

In [6]:
# Q2: Age
age_count = data.Q2.value_counts()
value_mask = data.Q2.isin(age_count.index[age_count < 180]) 
data.loc[value_mask,'Q2'] = "70+"

In [7]:
# Q3: Countries
# Categorize any country less than a threshold count in to other
countries_count = data.Q3.value_counts()
value_mask = data.Q3.isin(countries_count.index[countries_count < 100]) 
data.loc[value_mask,'Q3'] = "Other"
# Simplify Name
data['Q3'] = data['Q3'].replace('United States of America', 'USA')
data['Q3'] = data['Q3'].replace('United Kingdom of Great Britain and Northern Ireland', 'UKGBNI')
data['Q3'] = data['Q3'].replace('I do not wish to disclose my location', 'Other')

In [8]:
# Q4: Highest level of education
degree_count = data.Q4.value_counts(normalize=True)
# Simplify category name
data['Q4'] = data['Q4'].replace('Some college/university study without earning a bachelor’s degree', 'Non-conferred')
data['Q4'] = data['Q4'].replace('No formal education past high school', 'Secondary School')
# sns.catplot(x='Q4', y='Salary_label', data=data, kind="box").fig.set_size_inches(20,10)
# Since "Non-conferred" and "I prefer not to answer" show similar salary earnings, I can combine them
data['Q4'] = data['Q4'].replace('Non-conferred', 'Other')
data['Q4'] = data['Q4'].replace('I prefer not to answer', 'Other')
# sns.catplot(x='Q4', y='Salary_label', data=data, kind="box").fig.set_size_inches(20,10)

In [9]:
# Q5: Major in University/College
# There is 135 missing values
a = data[data['Q5'].isna()]
how_many_nan(data['Q5'])
# Simplified naming
data['Q5'] = data['Q5'].replace('A business discipline (accounting, economics, finance, etc.)', 'Business')
data['Q5'] = data['Q5'].replace('Information technology, networking, or system administration', 'Information technology')
data['Q5'] = data['Q5'].replace('Medical or life sciences (biology, chemistry, medicine, etc.)', 'Medical/Life Science')
data['Q5'] = data['Q5'].replace('Social sciences (anthropology, psychology, sociology, etc.)', 'Social Science')
data['Q5'] = data['Q5'].replace('Humanities (history, literature, philosophy, etc.)', 'Humanities')
# Grouped together non-STEM with others
data['Q5'] = data['Q5'].replace('I never declared a major', 'Other')
data['Q5'] = data['Q5'].replace('Fine arts or performing arts', 'Other')
data['Q5'] = data['Q5'].replace('Humanities', 'Other')
# sns.catplot(x='Q5', y='Salary_label', data=data, kind="box").fig.set_size_inches(60,10)

In [10]:
# Q6: Position Title
# Eliminate position titles less than a threshold
title_count = data.Q6.value_counts()
value_mask = data.Q6.isin(title_count.index[title_count < 100]) 
data.loc[value_mask,'Q6'] = "Other"

In [11]:
# Q7: Industry
industry_count = data.Q7.value_counts(normalize=True)

Computers/Technology                      0.270854
I am a student                            0.145700
Academics/Education                       0.128395
Accounting/Finance                        0.076544
Other                                     0.048610
Online Service/Internet-based Services    0.046341
Medical/Pharmaceutical                    0.038369
Government/Public Service                 0.033962
Insurance/Risk Assessment                 0.029425
Manufacturing/Fabrication                 0.028194
Marketing/CRM                             0.025018
Retail/Sales                              0.023073
Energy/Mining                             0.022360
Broadcasting/Communications               0.020675
Online Business/Internet-based Sales      0.017629
Shipping/Transportation                   0.015361
Non-profit/Service                        0.009916
Hospitality/Entertainment/Sports          0.009852
Military/Security/Defense                 0.009722
Name: Q7, dtype: float64

In [12]:
# Q8: Years of Experience
# There's 76 missing values - Should just remove it
how_many_nan(data['Q8'])
# Recategorize YOE
data['Q8'] = data['Q8'].replace('0-1', '0-3') # 0-3 years
data['Q8'] = data['Q8'].replace('1-2', '0-3')
data['Q8'] = data['Q8'].replace('2-3', '0-3')
data['Q8'] = data['Q8'].replace('3-4', '3-5') # 3-5 years
data['Q8'] = data['Q8'].replace('4-5', '3-5')
data['Q8'] = data['Q8'].replace('20-25', '20-30') # 20-30 years
data['Q8'] = data['Q8'].replace('25-30', '20-30')
data.Q8.value_counts(normalize=True)

0-3      0.564710
3-5      0.159773
5-10     0.128509
10-15    0.073145
15-20    0.038755
20-30    0.025728
30 +     0.009379
Name: Q8, dtype: float64

In [13]:
# Q10: Does current employer incorporate ML into their business
# There's 292 missing values
how_many_nan(data['Q10'])

292

In [14]:
def remove_words(col_name, unwanted_words):
    pat = r'\b(?:{})\b'.format('|'.join(unwanted_words))
    data[col_name] = data[col_name].str.replace(pat, '')
    data[col_name] = data[col_name].str.strip()

In [15]:
# Clean up Q40
Q40_words = ['Independent', 'projects', 'are', 'important', 'than', 'academic', 'achievements']
remove_words('Q40', Q40_words)

### Encode Features

In [16]:
class combine_col:
    # Function to concatenate multiple columns of texts in to one single column
    def sum_int(col_name, start, end):
        one = data[col_name+'_Part_'+str(start)]
        two = data[col_name+'_Part_'+str(start+1)]
        col_combined = one+two
        # iterate from start to end
        for i in range(start+2, end+1):
            next_col = data[col_name+'_Part_'+str(i)]
            col_combined = col_combined+next_col
        # replace zero with NaN
        col_combined = col_combined.replace(0, np.nan)
        return col_combined
    # Function to concatenate multiple columns of texts in to one single column
    def join_text(col_name, start, end):
        one = data[col_name+'_Part_'+str(start)]
        two = data[col_name+'_Part_'+str(start+1)]
        col_combined = one.astype(str)+' '+two
        # iterate from start to end
        for i in range(start+2, end+1):
            next_col = data[col_name+'_Part_'+str(i)]
            col_combined = col_combined.astype(str)+' '+next_col
        # Replace empty cells with NaN
        col_combined = col_combined.str.strip() # remove leading/trailing space in a column
        col_combined = col_combined.replace(r'^\s*$', regex=True).replace('',np.nan)
        return col_combined

In [17]:
# Class for filling in information
class fill_info:
    # Function to fill NaN values in multiple columns with 0
    def nan2zero(col_name, start, end):
        for i in range (start, end+1):
            data[col_name+'_Part_'+str(i)] = data[col_name+'_Part_'+str(i)].fillna(0)
    # Function to fill all text values with 1b
    def text2one(col_name, start, end):
        # Before converting all texts to 1, save a list of the categories
        features_list = list()
        for i in range (start, end+1): #text2one
            spot = data[col_name+'_Part_'+str(i)].ne(0).idxmax()
            feature = data[col_name+'_Part_'+str(i)].iloc[spot-1]
            features_list.append(feature)
        for i in range (start, end+1):
            data[col_name+'_Part_'+str(i)] = data[col_name+'_Part_'+str(i)].astype(bool).astype(int)
        return features_list
    # Function to fill NaN values in multiple columns with space
    def nan2space(col_name, start, end):
        for i in range (start, end+1):
            data[col_name+'_Part_'+str(i)] = data[col_name+'_Part_'+str(i)].fillna(' ')
    # Combination
    def nan_text(col_name,start, end):
        for i in range (start, end+1): #nan2zero
            data[col_name+'_Part_'+str(i)] = data[col_name+'_Part_'+str(i)].fillna(0)
        # Before converting all texts to 1, save a list of the categories
        features_list = list()
        for i in range (start, end+1): #text2one
            spot = data[col_name+'_Part_'+str(i)].ne(0).idxmax()
            feature = data[col_name+'_Part_'+str(i)].iloc[spot-1]
            features_list.append(feature)
        for i in range (start, end+1): #text2one
            data[col_name+'_Part_'+str(i)] = data[col_name+'_Part_'+str(i)].astype(bool).astype(int)
        return features_list
    # Function for getting the features of every column before they are converted to numbers
    def get_features(col_name, start, end):
        features_list = list()
        for i in range (start, end+1):
            spot = data[col_name+'_Part_'+str(i)].ne(0).idxmax()
            feature = data[col_name+'_Part_'+str(i)].iloc[spot-1]
            features_list.append(feature)
        return features_list
    def col2int(col_name, start, end):
        for i in range (start, end+1): #fill nan with zero
            data[col_name+'_Part_'+str(i)] = data[col_name+'_Part_'+str(i)].fillna(0)
        for i in range (start, end+1): # convert everything to type float
            data[col_name+'_Part_'+str(i)] = data[col_name+'_Part_'+str(i)].astype(float)
        

##### Create dummy variables

In [18]:
def dummy(col_name):
    global data
    dummy = pd.get_dummies(data[col_name], prefix=col_name)
    data = data.drop([col_name], axis=1)
    return dummy

In [19]:
Q1_dummies = dummy('Q1')
Q2_dummies = dummy('Q2')
Q3_dummies = dummy('Q3')
Q4_dummies = dummy('Q4')
Q5_dummies = dummy('Q5')
Q6_dummies = dummy('Q6')
Q7_dummies = dummy('Q7')
Q8_dummies = dummy('Q8')
Q10_dummies = dummy('Q10')
Q12_dummies = dummy('Q12_MULTIPLE_CHOICE')
Q17_dummies = dummy('Q17')
Q18_dummies = dummy('Q18')
Q20_dummies = dummy('Q20')
Q22_dummies = dummy('Q22')
Q23_dummies = dummy('Q23')
Q24_dummies = dummy('Q24')
Q25_dummies = dummy('Q25')
Q26_dummies = dummy('Q26')
Q32_dummies = dummy('Q32')
Q37_dummies = dummy('Q37')
Q39P1_dummies = dummy('Q39_Part_1')
Q39P2_dummies = dummy('Q39_Part_2')
Q40_dummies = dummy('Q40')
Q41P1_dummies = dummy('Q41_Part_1')
Q41P2_dummies = dummy('Q41_Part_2')
Q41P3_dummies = dummy('Q41_Part_3')
Q43_dummies = dummy('Q43')
Q46_dummies = dummy('Q46')
Q48_dummies = dummy('Q48')

In [20]:
# Q34
fill_info.col2int('Q34', 1, 6)
# Q35
fill_info.col2int('Q35', 1, 6)

##### Encode categorical data

In [21]:
# Q11
# encode Q11 part 1-7 all nan's to zero, and all texts to 1
Q11_features = fill_info.nan_text('Q11', 1, 7)
# Combine columns of int (0 or 1) into a single column of sum (int)
Q11_combined = combine_col.sum_int('Q11', 1, 7)
# There are 904 missing responses for Q11
how_many_nan(Q11_combined)

904

In [22]:
# Q13
# encode Q13
Q13_features = fill_info.nan_text('Q13', 1, 15)
# Combine columns of int (0 or 1) into a single column of sum (int)
Q13_combined = combine_col.sum_int('Q13', 1, 15)
# 1630 missing values
how_many_nan(Q13_combined)

1630

In [23]:
# Q14
# encode Q14
Q14_features = fill_info.nan_text('Q14', 1, 11)
# Combine columns of int (0 or 1) into a single column of sum (int)
Q14_combined = combine_col.sum_int('Q14', 1, 11)
# 1708 missing values
how_many_nan(Q14_combined)

1708

In [24]:
# encode Q15
Q15_features = fill_info.nan_text('Q15', 1, 7)
# Combine columns of int (0 or 1) into a single column of sum (int)
Q15_combined = combine_col.sum_int('Q15', 1, 7)
# 1772 missing values - question includes choice for people to select "none"
how_many_nan(Q15_combined)

1772

In [25]:
# encode Q16
Q16_features = fill_info.nan_text('Q16', 1, 18)
# Combine columns of int (0 or 1) into a single column of sum (int)
Q16_combined = combine_col.sum_int('Q16', 1, 18)
# 1798 missing values
how_many_nan(Q16_combined)

1798

In [26]:
# encode Q19
Q19_features = fill_info.nan_text('Q19', 1, 19)
# Combine columns of int (0 or 1) into a single column of sum (int)
Q19_combined = combine_col.sum_int('Q19', 1, 19)
# 1874 missing values
how_many_nan(Q19_combined)

1874

In [27]:
# encode Q21
Q21_features = fill_info.nan_text('Q21', 1, 13)
# Combine columns of int (0 or 1) into a single column of sum (int)
Q21_combined = combine_col.sum_int('Q21', 1, 13)
# 1935 missing values
how_many_nan(Q21_combined)

1935

In [28]:
# encode Q27
Q27_features = fill_info.nan_text('Q27', 1, 20)
# Combine columns of int (0 or 1) into a single column of sum (int)
Q27_combined = combine_col.sum_int('Q27', 1, 20)
# 6776 missing values
how_many_nan(Q27_combined)

6776

In [29]:
# encode Q28
Q28_features = fill_info.nan_text('Q28', 1, 43)
# Combine columns of int (0 or 1) into a single column of sum (int)
Q28_combined = combine_col.sum_int('Q28', 1, 43)
# 6900 missing values
how_many_nan(Q28_combined)

6900

In [30]:
# encode Q29
Q29_features = fill_info.nan_text('Q29', 1, 28)
# Combine columns of int (0 or 1) into a single column of sum (int)
Q29_combined = combine_col.sum_int('Q29', 1, 28)
# 7022 missing values
how_many_nan(Q29_combined)

7022

In [31]:
# encode Q30
Q30_features = fill_info.nan_text('Q30', 1, 25)
# Combine columns of int (0 or 1) into a single column of sum (int)
Q30_combined = combine_col.sum_int('Q30', 1, 25)
# 7764 missing values
how_many_nan(Q30_combined)

7764

In [32]:
# encode Q31
Q31_features = fill_info.nan_text('Q31', 1, 12)
# Combine columns of int (0 or 1) into a single column of sum (int)
Q31_combined = combine_col.sum_int('Q31', 1, 12)
# 3011 missing values
how_many_nan(Q31_combined)

3011

In [33]:
# encode Q33
Q33_features = fill_info.nan_text('Q33', 1, 11)
# Combine columns of int (0 or 1) into a single column of sum (int)
Q33_combined = combine_col.sum_int('Q33', 1, 11)
# 3085 missing values
how_many_nan(Q33_combined)

3085

In [34]:
# encode Q36
Q36_features = fill_info.nan_text('Q36', 1, 13)
# Combine columns of int (0 or 1) into a single column of sum (int)
Q36_combined = combine_col.sum_int('Q36', 1, 13)
# 3834 missing values
how_many_nan(Q36_combined)

3834

In [35]:
# encode Q38
Q38_features = fill_info.nan_text('Q38', 1, 22)
# Combine columns of int (0 or 1) into a single column of sum (int)
Q38_combined = combine_col.sum_int('Q38', 1, 22)
# 3459 missing values
how_many_nan(Q38_combined)

3459

In [36]:
# encode Q42
Q42_features = fill_info.nan_text('Q42', 1, 5)
# Combine columns of int (0 or 1) into a single column of sum (int)
Q42_combined = combine_col.sum_int('Q42', 1, 5)
# 5306 missing values
how_many_nan(Q42_combined)

5306

In [37]:
# encode Q44
Q44_features = fill_info.nan_text('Q44', 1, 6)
# Combine columns of int (0 or 1) into a single column of sum (int)
Q44_combined = combine_col.sum_int('Q44', 1, 6)
# 5469 missing values
how_many_nan(Q44_combined)

5469

In [38]:
# encode Q45
Q45_features = fill_info.nan_text('Q45', 1, 6)
# Combine columns of int (0 or 1) into a single column of sum (int)
Q45_combined = combine_col.sum_int('Q45', 1, 6)
# 5226 missing values
how_many_nan(Q45_combined)

5226

In [39]:
# encode Q47
Q47_features = fill_info.nan_text('Q47', 1, 16)
# Combine columns of int (0 or 1) into a single column of sum (int)
Q47_combined = combine_col.sum_int('Q47', 1, 16)
# 5382 missing values
how_many_nan(Q47_combined)

5382

In [40]:
# encode Q49
Q49_features = fill_info.nan_text('Q49', 1, 12)
# Combine columns of int (0 or 1) into a single column of sum (int)
Q49_combined = combine_col.sum_int('Q49', 1, 12)
# 5718 missing values
how_many_nan(Q49_combined)

5718

In [41]:
# encode Q50
Q50_features = fill_info.nan_text('Q50', 1, 8)
# Combine columns of int (0 or 1) into a single column of sum (int)
Q50_combined = combine_col.sum_int('Q50', 1, 8)
# 5767 missing values
how_many_nan(Q50_combined)

5767

#### Cleaned Data

In [42]:
# concatenate all dummy data
dummy_data = pd.concat([Q1_dummies, Q2_dummies, Q3_dummies, Q4_dummies, Q5_dummies, Q6_dummies, Q7_dummies, Q8_dummies, Q10_dummies, Q12_dummies, Q17_dummies, Q18_dummies, Q20_dummies, Q22_dummies, Q23_dummies, Q24_dummies, Q25_dummies, Q26_dummies, Q32_dummies, Q37_dummies, Q39P1_dummies, Q39P2_dummies, Q40_dummies, Q41P1_dummies, Q41P2_dummies, Q41P3_dummies, Q43_dummies, Q46_dummies, Q48_dummies], axis=1)
# drop any column with "TEXT" in title
data.drop([col for col in data.columns if 'TEXT' in col],axis=1,inplace=True)
# drop other unecessary columns
data.drop(['Unnamed: 0', 'Time from Start to Finish (seconds)', 'index'], axis=1, inplace=True)
# form cleaned data
cleaned_data = pd.concat([dummy_data, data], axis=1)

### Logistic Regression

In [43]:
X = cleaned_data.drop(['Q9'],axis=1)
y = cleaned_data['Q9']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.3)

In [45]:
import warnings
#warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore")
model = LogisticRegression()    
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [48]:
model = LogisticRegression()
scaler = StandardScaler()
kfold = KFold(n_splits=10)
kfold.get_n_splits(X)

accuracy = np.zeros(10)
np_idx = 0

for train_idx, test_idx in kfold.split(X):
    X_train, X_test = X.values[train_idx], X.values[test_idx]
    y_train, y_test = y.values[train_idx], y.values[test_idx]
    
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    model.fit(X_train, y_train)
    
    predictions = model.predict(X_test)
    
    TN = confusion_matrix(y_test, predictions)[0][0]
    FP = confusion_matrix(y_test, predictions)[0][1]
    FN = confusion_matrix(y_test, predictions)[1][0]
    TP = confusion_matrix(y_test, predictions)[1][1]
    total = TN + FP + FN + TP
    ACC = (TP + TN) / float(total)
    
    accuracy[np_idx] = ACC*100
    np_idx += 1
    
    print ("Fold {}: Accuracy: {}%".format(np_idx, round(ACC,3)))   

print ("Average Score: {}%({}%)".format(round(np.mean(accuracy),3),round(np.std(accuracy),3)))

Fold 1: Accuracy: 0.784%
Fold 2: Accuracy: 0.73%
Fold 3: Accuracy: 0.711%
Fold 4: Accuracy: 0.747%
Fold 5: Accuracy: 0.75%
Fold 6: Accuracy: 0.746%
Fold 7: Accuracy: 0.771%
Fold 8: Accuracy: 0.767%
Fold 9: Accuracy: 0.737%
Fold 10: Accuracy: 0.772%
Average Score: 75.161%(2.114%)
